In [1]:
import json

with open('config.json', 'r') as f:
    config = json.load(f)

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA

In [3]:
df_news_clusterizada = pd.read_csv(config["DF_ITEMS_CLUSTERED_FEATURES"])
df_news_clusterizada.drop(columns=["Unnamed: 0","Unnamed: 0.1"],inplace=True)

In [4]:
df_news_clusterizada.head(2)

,page,url,issued,modified,title,body,caption,cleaned_body,cleaned_title,cleaned_caption,combined_text,cluster,word_count
0,7371a9b5-5824-4c57-8704-00a74feebe79,http://g1.globo.com/al/alagoas/noticia/2018/09...,2018-09-13 14:52:55+00:00,2018-09-14 16:14:49+00:00,Corpo de motorista da Uber é encontrado em can...,Corpo de motorista de aplicativo desaparecido ...,"Segundo a polícia, Antônio Vitor foi solicitad...",corpo motorista aplicativo desaparecido encont...,corpo motorista uber encontrado canavial rotei...,polícia antônio vitor solicitado corridas desa...,corpo motorista uber encontrado canavial rotei...,0,234
1,7a5ea08f-4583-49e2-ba52-a71999443f7b,http://g1.globo.com/am/amazonas/noticia/detent...,2018-05-20 20:42:40+00:00,2018-05-20 20:42:40+00:00,Detento recapturado após fuga por túnel volta ...,Detento disse que passou nome falso ao dar ent...,Ele tinha registro em presídio com nome falso....,detento disse passou nome falso entrada presíd...,detento recapturado fuga túnel volta cdpm manaus,tinha registro presídio nome falso presos esca...,detento recapturado fuga túnel volta cdpm mana...,0,345


In [5]:
import numpy as np

# Convert the string column to datetime
df_news_clusterizada["data"] = pd.to_datetime(df_news_clusterizada["issued"])

In [6]:
df_news_clusterizada.head(2)

,page,url,issued,modified,title,body,caption,cleaned_body,cleaned_title,cleaned_caption,combined_text,cluster,word_count,data
0,7371a9b5-5824-4c57-8704-00a74feebe79,http://g1.globo.com/al/alagoas/noticia/2018/09...,2018-09-13 14:52:55+00:00,2018-09-14 16:14:49+00:00,Corpo de motorista da Uber é encontrado em can...,Corpo de motorista de aplicativo desaparecido ...,"Segundo a polícia, Antônio Vitor foi solicitad...",corpo motorista aplicativo desaparecido encont...,corpo motorista uber encontrado canavial rotei...,polícia antônio vitor solicitado corridas desa...,corpo motorista uber encontrado canavial rotei...,0,234,2018-09-13 14:52:55+00:00
1,7a5ea08f-4583-49e2-ba52-a71999443f7b,http://g1.globo.com/am/amazonas/noticia/detent...,2018-05-20 20:42:40+00:00,2018-05-20 20:42:40+00:00,Detento recapturado após fuga por túnel volta ...,Detento disse que passou nome falso ao dar ent...,Ele tinha registro em presídio com nome falso....,detento disse passou nome falso entrada presíd...,detento recapturado fuga túnel volta cdpm manaus,tinha registro presídio nome falso presos esca...,detento recapturado fuga túnel volta cdpm mana...,0,345,2018-05-20 20:42:40+00:00


In [7]:
# Find the most recent date
most_recent_date = df_news_clusterizada["data"].max()

# Calculate the difference in days
df_news_clusterizada["idade_em_dias"] = (most_recent_date - df_news_clusterizada["data"]).dt.days

In [24]:
most_recent_date

Timestamp('2022-08-15 02:57:16+0000', tz='UTC')

In [8]:
df_news_clusterizada["idade_em_dias"].describe()

count    13809.000000
mean       425.284380
std        511.855106
min          0.000000
25%         67.000000
50%        206.000000
75%        592.000000
max       2246.000000
Name: idade_em_dias, dtype: float64

In [9]:
# max_news_age = df_news_clusterizada["idade_em_dias"].max()
max_news_age = 3000
settling_time_taus_factor = 6
time_constant = max_news_age/settling_time_taus_factor
print(max_news_age, time_constant)

3000 500.0


In [10]:
# Assign weights using exponential decay (you can adjust the base for scaling)
df_news_clusterizada["peso"] = np.exp(-df_news_clusterizada["idade_em_dias"] / time_constant) 

In [11]:
# Adjust range to [0.5, 1]
new_min = 0.1
new_max = 1
df_news_clusterizada["peso_ajustado"] = df_news_clusterizada["peso"] * (new_max - new_min) + new_min


In [12]:
df_news_clusterizada.head(2)

,page,url,issued,modified,title,body,caption,cleaned_body,cleaned_title,cleaned_caption,combined_text,cluster,word_count,data,idade_em_dias,peso,peso_ajustado
0,7371a9b5-5824-4c57-8704-00a74feebe79,http://g1.globo.com/al/alagoas/noticia/2018/09...,2018-09-13 14:52:55+00:00,2018-09-14 16:14:49+00:00,Corpo de motorista da Uber é encontrado em can...,Corpo de motorista de aplicativo desaparecido ...,"Segundo a polícia, Antônio Vitor foi solicitad...",corpo motorista aplicativo desaparecido encont...,corpo motorista uber encontrado canavial rotei...,polícia antônio vitor solicitado corridas desa...,corpo motorista uber encontrado canavial rotei...,0,234,2018-09-13 14:52:55+00:00,1431,0.057154,0.151439
1,7a5ea08f-4583-49e2-ba52-a71999443f7b,http://g1.globo.com/am/amazonas/noticia/detent...,2018-05-20 20:42:40+00:00,2018-05-20 20:42:40+00:00,Detento recapturado após fuga por túnel volta ...,Detento disse que passou nome falso ao dar ent...,Ele tinha registro em presídio com nome falso....,detento disse passou nome falso entrada presíd...,detento recapturado fuga túnel volta cdpm manaus,tinha registro presídio nome falso presos esca...,detento recapturado fuga túnel volta cdpm mana...,0,345,2018-05-20 20:42:40+00:00,1547,0.045320,0.140788


In [13]:
df_news_clusterizada.describe()

,cluster,word_count,idade_em_dias,peso,peso_ajustado
count,13809.000000,13809.000000,13809.000000,13809.000000,13809.000000
mean,0.008473,489.183866,425.284380,0.590367,0.631331
std,0.091660,397.461229,511.855106,0.319329,0.287396
min,0.000000,5.000000,0.000000,0.011198,0.110078
25%,0.000000,240.000000,67.000000,0.306052,0.375447
50%,0.000000,384.000000,206.000000,0.662324,0.696092
75%,0.000000,613.000000,592.000000,0.874590,0.887131
max,1.000000,10053.000000,2246.000000,1.000000,1.000000


In [14]:
df_news_clusterizada.to_csv(config["DF_ITEMS_CLUSTERED_FEATURES_ADJ"])